In [72]:
# A dependency of the preprocessing for BERT inputs
!pip install -q -U "tensorflow-text==2.11.*"

In [73]:
!pip install -q tf-models-official==2.11.0

In [74]:
import re
import string
import random
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization # to create AdamW optimizer

import matplotlib.pyplot as plt

tf.get_logger().setLevel('ERROR')

In [75]:
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,Review,Rating
0,Great game when i was a kid I grew up watching...,5
1,Nice game,1
2,Balo game but bholo laguchi khelibaku gote khu...,5
3,This is the worst computer cheating game I've ...,1
4,Great,5


In [76]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,Review,Rating
2312,PUBG is the best becuse I can easily get a mic...,5
16845,Very good game,5
6105,I love the fighting of the game and the mechan...,5
18082,Ahem listen to me nowwww babeee,1
11335,It's lots of fun though,5


In [77]:
train_df["Rating"].value_counts()

5    12376
1     4120
4     2233
3     1369
2      902
Name: Rating, dtype: int64

In [78]:
train_dataset = np.array(train_df["Review"])

In [79]:
train_data = tf.data.Dataset.from_tensor_slices(train_dataset)
train_data.prefetch(buffer_size=tf.data.AUTOTUNE)

<PrefetchDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [80]:
train_labels = np.array(train_df["Rating"])

In [87]:
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
encoder_inputs = preprocessor(text_input)
encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4", trainable=True)
outputs = encoder(encoder_inputs)
pooled_output = outputs["pooled_output"]
net = tf.keras.layers.Dropout(0.1)(pooled_output)
net = tf.keras.layers.Dense(6, activation="softmax", name='classifier')(net)
model = tf.keras.Model(text_input, net)

In [88]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
metrics = "accuracy"

In [89]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_data).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [90]:
model.compile(optimizer=optimizer,
              loss=loss,
              metrics=metrics)

In [ ]:
history = model.fit(train_dataset,
                    train_labels,
                    epochs=20,
                    validation_split=0.1)

Epoch 1/20
 41/591 [=>............................] - ETA: 8:03 - loss: 2.1249 - accuracy: 0.0732